# 株価取得処理

In [2]:
# ライブラリのインストール
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 15.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.3/173.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.7/157.7 kB 11.3 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.18.1-cp310-cp310-macosx_12_0_arm64.whl size=285060 sha256=2aa1381c74eaa9e1baa00e4f3421038c8e2233b6987fc2e3cb2a3b405853ad46
  Stored in directory: /Users/ymto/Library/Caches/pip/wheels/8d/c2/af/d591c26196f80ecdedb79664b51642ff1cf49af64d5cd69999

In [3]:
import yfinance as yf
import pandas as pd

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
hist = ticker.history(period="1y")
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE')}")
print(f"PBR (株価純資産倍率): {info.get('priceToBook')}")
print(f"配当利回り: {info.get('dividendYield')}")

# 3. 配当性向の取得
# infoの中に'payoutRatio'というキーで配当性向が含まれている
print("\n--- 3. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    print(f"配当性向: {payout_ratio:.2%}") # パーセント表示
else:
    print("配当性向データがありません。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-09 00:00:00+09:00  2670.0  2672.0  2637.0  2640.0  13584200        0.0           0.0
2025-06-10 00:00:00+09:00  2653.0  2663.5  2640.0  2657.5  19556800        0.0           0.0
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2555.0
PER (株価収益率): 7.107291
PBR (株価純資産倍率): 0.9280495
配当利回り: 3.72

--- 3. 配当性向 ---
配当性向: 25.03%
